In [1]:
import os
from os import listdir
from os.path import isfile, join
import glob
import string
import operator
import csv
import pandas as pd
import calendar
import numpy as np

In [2]:
homedir = os.path.expanduser("~")
path_to_data = homedir+"\\Assignment 3\\Data"
print(path_to_data)

C:\Users\priya\Assignment 3\Data


### Getting the data from CSV 

In [3]:
output_dist={}
for subdir,dirs, files in os.walk(path_to_data):
    #print(files)
    for file in files:
        if file == 'employee_compensation.csv':
            #print(file)
            csv_data1=pd.read_csv(os.path.join(subdir,file))
print(csv_data1.head(2))

Year Type  Year  Organization Group Code  \
0  Calendar  2013                        2   
1    Fiscal  2015                        4   

                        Organization Group Department Code  \
0  Public Works, Transportation & Commerce             AIR   
1                         Community Health             DPH   

           Department  Union Code                               Union  \
0  Airport Commission        38.0  Plumbers and Pipefitters, Local 38   
1       Public Health       790.0    SEIU - Miscellaneous, Local 1021   

  Job Family Code        Job Family         ...         Employee Identifier  \
0            7300  Journeyman Trade         ...                       52206   
1            2600    Dietary & Food         ...                       52950   

   Salaries  Overtime  Other Salaries  Total Salary  Retirement  \
0  98528.52      0.00         3955.59     102484.11    20142.55   
1  69169.82    755.02         4179.08      74103.92    15641.92   

   Health/Dental  Other Benefits  Total Benefits  Total Compensation  
0       12918.24         8371.76        41432.55           143916.66  
1       12499.30         5669.02        33810.24           107914.16  

[2 rows x 22 columns]

### Getting only relevant columns  

In [4]:
employee_df = csv_data1[["Year Type","Employee Identifier","Job Family","Salaries","Overtime","Total Benefits","Total Compensation"]].copy()
print(employee_df.head(2))

Year Type  Employee Identifier        Job Family  Salaries  Overtime  \
0  Calendar                52206  Journeyman Trade  98528.52      0.00   
1    Fiscal                52950    Dietary & Food  69169.82    755.02   

   Total Benefits  Total Compensation  
0        41432.55           143916.66  
1        33810.24           107914.16

# getting only Employees for  calender year

In [5]:
employee_df = employee_df[(employee_df['Year Type'] == 'Calendar')]
print(employee_df.head(5))

Year Type  Employee Identifier                     Job Family  Salaries  \
0  Calendar                52206               Journeyman Trade  98528.52   
2  Calendar                33958                     Management    200.00   
3  Calendar                33073                  Legal & Court   1783.00   
5  Calendar                23930  Clerical, Secretarial & Steno  80207.24   
7  Calendar                38054            Public Service Aide  24180.00   

   Overtime  Total Benefits  Total Compensation  
0      0.00        41432.55           143916.66  
2      0.00           67.28              267.28  
3      0.00          736.13             2519.13  
5   2232.53        35843.78           119851.55  
7      0.00        11956.70            36136.70

### Mean of total compansation

In [6]:
employee_df_total_com = employee_df.groupby('Employee Identifier', as_index=False)['Total Compensation'].mean()
print(employee_df_total_com.head(5))

Employee Identifier  Total Compensation
0                    1        60577.913333
1                    2        15741.910000
2                    3        21237.550000
3                    4        26526.275000
4                    5        22415.950000

### Calculating mean for total benefits 

In [7]:
employee_df_total_benifit = employee_df.groupby('Employee Identifier', as_index=False)['Total Benefits'].mean()
print(employee_df_total_benifit.head(5))

Employee Identifier  Total Benefits
0                    1    20213.936667
1                    2     4680.210000
2                    3     6550.720000
3                    4     7741.395000
4                    5     2885.545000

# Joining both frames to get final frame with mean of both columns 

In [8]:
final_df = pd.merge(employee_df_total_com,employee_df_total_benifit,on ='Employee Identifier' , how='inner')
final_df.columns = ["Employee Identifier","Total Compensation","Total Benefits"]
print(final_df.head(5))

Employee Identifier  Total Compensation  Total Benefits
0                    1        60577.913333    20213.936667
1                    2        15741.910000     4680.210000
2                    3        21237.550000     6550.720000
3                    4        26526.275000     7741.395000
4                    5        22415.950000     2885.545000

### Getting data for employees whos overtime is more than 5% of salaries 

In [9]:
salaries_df = employee_df[(employee_df['Overtime'] > 0.05*employee_df['Salaries'])]
print(salaries_df.head(5))

Year Type  Employee Identifier                     Job Family   Salaries  \
14  Calendar                23029     Agriculture & Horticulture   63978.00   
23  Calendar                20763  Administrative-Labor & Trades  133977.95   
38  Calendar                 8705               Journeyman Trade   86530.02   
40  Calendar                13888                Police Services  117139.22   
44  Calendar                38574                  Fire Services  130341.65   

    Overtime  Total Benefits  Total Compensation  
14  17039.61        34064.57           117266.41  
23  20699.15        48600.84           203277.94  
38  17962.95        40912.58           155960.76  
40  20576.29        37835.64           181298.64  
44  29607.28        46842.09           223083.84

In [10]:
job_family_df = salaries_df[["Job Family","Total Benefits","Total Compensation"]].copy()

Job Family  Total Benefits  Total Compensation
14     Agriculture & Horticulture        34064.57           117266.41
23  Administrative-Labor & Trades        48600.84           203277.94
38               Journeyman Trade        40912.58           155960.76
40                Police Services        37835.64           181298.64
44                  Fire Services        46842.09           223083.84

# Calculating the mean of both columns 

In [11]:
job_family_total_benifits = job_family_df.groupby('Job Family', as_index=False)['Total Benefits'].mean()
print(job_family_total_benifits.head(5))

Job Family  Total Benefits
0         Administrative-DPW/PUC    51357.650000
1  Administrative-Labor & Trades    40301.818718
2     Agriculture & Horticulture    29578.258462
3              Airport Operation    34531.437719
4  Budget, Admn & Stats Analysis    26404.129524

In [12]:
job_family_total_comp = job_family_df.groupby('Job Family', as_index=False)['Total Compensation'].mean()
print(job_family_total_comp.head(5))

Job Family  Total Compensation
0         Administrative-DPW/PUC       211601.430000
1  Administrative-Labor & Trades       183350.857436
2     Agriculture & Horticulture       100279.346697
3              Airport Operation       126629.480938
4  Budget, Admn & Stats Analysis        91785.631667

## Joining both data frames 

In [13]:
final_job_family_df = pd.merge(job_family_total_benifits,job_family_total_comp,on ='Job Family' , how='inner')
final_job_family_df.columns = ["Job Family","Total Benefits","Total Compensation"]
print(final_job_family_df.head(5))

Job Family  Total Benefits  Total Compensation
0         Administrative-DPW/PUC    51357.650000       211601.430000
1  Administrative-Labor & Trades    40301.818718       183350.857436
2     Agriculture & Horticulture    29578.258462       100279.346697
3              Airport Operation    34531.437719       126629.480938
4  Budget, Admn & Stats Analysis    26404.129524        91785.631667

In [14]:
final_job_family_df["Percentage"] = (final_job_family_df['Total Benefits']/ final_job_family_df['Total Compensation'])*100
print(final_job_family_df.head(20))

Job Family  Total Benefits  Total Compensation  \
0           Administrative-DPW/PUC    51357.650000       211601.430000   
1    Administrative-Labor & Trades    40301.818718       183350.857436   
2       Agriculture & Horticulture    29578.258462       100279.346697   
3                Airport Operation    34531.437719       126629.480938   
4    Budget, Admn & Stats Analysis    26404.129524        91785.631667   
5    Clerical, Secretarial & Steno    11203.637916        37675.828898   
6            Community Development      229.670000          735.110000   
7   Computer Operatns & Repro Svcs    22222.893901        73466.205461   
8          Construction Inspection    34578.044644       130123.717865   
9           Correction & Detention    35272.274437       153870.789226   
10                  Dietary & Food    18344.154662        61278.415288   
11              Emergency Services    12765.360000        44038.855000   
12            Energy & Environment     3309.920000        12759.420000   
13                   Fire Services    41434.131988       203525.467851   
14  Health & Sanitation Inspection    38702.501818       141710.384545   
15         Hospital Administration    31508.771795       107011.031667   
16          Housekeeping & Laundry    27542.950639        89353.140639   
17                  Human Services    30504.521495       107980.903442   
18             Information Systems    25485.276618        90188.009853   
19                Journeyman Trade    36772.678593       142543.271193   

    Percentage  
0    24.270937  
1    21.980709  
2    29.495863  
3    27.269667  
4    28.767171  
5    29.736938  
6    31.242943  
7    30.249138  
8    26.573207  
9    22.923308  
10   29.935752  
11   28.986585  
12   25.940991  
13   20.358205  
14   27.310985  
15   29.444415  
16   30.824827  
17   28.249922  
18   28.257943  
19   25.797555

In [20]:
def save_to_pdf():
    if not os.path.exists(homedir+"\\Assignment 3\\Output\\Question2_part2"):
        os.makedirs(homedir+"\\Assignment 3\\Output\\Question2_part2")
    with open(homedir+'\\Assignment 3\\Output\\Question2_part2\\Q2P2_ouptput.csv', 'w') as myfile:
        myfile.write(final_job_family_df.to_csv())

In [ ]:
save_to_pdf()